In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from pyomo.opt import SolverFactory, TerminationCondition
from src.analyses.value_function import ValueFunction
from src.analyses.convergence_analysis import HausdorffAnalyzer
import numpy as np
import matplotlib.pyplot as plt
from src.models.cz_model import CaoZavalaModel, CaoZavalaAlgo
from scipy.optimize import curve_fit
from NSPLIB.src.instances.ex8_4_4.ex8_4_4_SP import const_model
import copy
# Constants
ex844_obj= 0.33272388311799445
ex844_sol = {
    'x10': -0.23129676903369037, 'x11': 1.2249985914391361, 'x12': 0.05263738880099749, 'x13': 0.5614202118628403,
    'x14': 0.6217131656002401, 'x15': 1.1, 'x16': 0.806868640168786, 'x17': 0.663161467338442, 'x6': -1.1370129865018144,
    'x7': 1.5979758364259327, 'x8': -0.5986905053355759, 'x9': 1.372365581205024
}
ex844_y_bound = {
    'x10': [-1.2, 0.8], 'x11': [0.1, 2.1], 'x12': [-1.1, 0.9], 'x13': [0, 1],
    'x14': [0, 1], 'x15': [1.1, 1.3], 'x16': [0, 1], 'x17': [0, 1],
    'x6': [-2, 0], 'x7': [0.5, 2.5], 'x8': [-1.5, 0.5], 'x9': [0.2, 2.2]
}
ex844_y_bound = {
    'x10': [-0.25, -0.2], 'x11': [1.2, 1.24], 'x12': [0.05, 0.06], 'x13': [0.5, 0.6],
    'x14': [0.6, 0.7], 'x15': [1.1, 1.11], 'x16': [0.8, 0.85], 'x17': [0.6, 0.7],
    'x6': [-1.15, -1.1], 'x7': [1.55, 1.65], 'x8': [-0.7, -0.5], 'x9': [1.3, 1.4]
}
# Build model
sto_m = const_model()
m = CaoZavalaModel.from_sto_m(sto_m)
m.build()

v_f= ValueFunction(m,solver="baron")
y_dimension = {
    'x10': {}, 'x11': {}, 'x12': {}, 'x13': {},
    'x14': {}, 'x15': {}, 'x16': {}, 'x17': {},
    'x6': {}, 'x7': {}, 'x8': {}, 'x9': {}
}
steps=20
for y in y_dimension.keys():
    v_f.calc_1D(idx=y, y_val_fix=copy.deepcopy(ex844_sol),
                    interval=ex844_y_bound[y], step=steps)
    for s in m.scenarios:
        y_dimension[y][s] = v_f.value_func[s]
    y_dimension[y]['total'] = v_f.total_value_func


# Specify grid dimensions
n_rows = len(y_dimension)
n_cols = len(m.scenarios) + 1  # +1 for 'total'

# Create a grid of Output widgets for the plots
outputs = [[widgets.Output() for _ in range(n_cols)] for _ in range(n_rows)]

# Fill each plot with the corresponding value function data
for i, y_key in enumerate(y_dimension.keys()):
    for j in range(n_cols):
        with outputs[i][j]:
            x = np.linspace(ex844_y_bound[y_key][0], ex844_y_bound[y_key][1], len(y_dimension[y_key]['total']))
            if j < len(m.scenarios):
                scenario_key = list(m.scenarios)[j]
                y_vals = y_dimension[y_key][scenario_key]
                title = f"{y_key} - Scenario {scenario_key}"
            else:
                y_vals = y_dimension[y_key]['total']
                title = f"{y_key} - Total"
            fig, ax = plt.subplots(figsize=(4,4))
            ax.plot(x, y_vals,marker="o")
            ax.set_title(title)
            ax.set_xlabel(y_key)
            ax.set_ylabel("Objective Value")
            ax.tick_params(axis='both', labelsize=14)
            ax.xaxis.label.set_size(16)
            ax.yaxis.label.set_size(16)
            ax.title.set_size(18)
            plt.tight_layout()
            plt.show()

# Arrange the outputs in a grid
grid = widgets.GridBox(
    [outputs[i][j] for i in range(n_rows) for j in range(n_cols)],
    layout=widgets.Layout(grid_template_columns=f"repeat({n_cols}, 400px)")
)

display(grid)

GridBox(children=(Output(), Output(), Output(), Output(), Output(), Output(), Output(), Output(), Output(), Ou…